In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from collections import namedtuple
from random import shuffle
from itertools import groupby
import datetime as dt

In [48]:
import gensim.models.doc2vec as d2v
import numpy as np
import cvxpy as cvx

In [4]:
from articles import Articles
from helper.collections import Namedtuples
import model.utility as ut
import model.problem as pr

In [5]:
Sample = namedtuple('Sample',['date','data','logreturn'])
DateReturn = namedtuple('DateReturn',['date','logreturn'])

In [6]:
sp500 = np.load('sp500.npy')
sp500 = [DateReturn(*record) for record in sp500]
sp500 = sorted(sp500,key=lambda r: r.date)

In [7]:
all_articles = list(Articles(['id','date']))
all_articles = [a._replace(date=a.date.date()) for a in all_articles] # Keep only the date, dismiss the rest of time
all_articles = sorted(all_articles,key=lambda a: a.date)

In [8]:
model = d2v.Doc2Vec.load('300model')

In [9]:
def create_sample(date,daily_articles):
    ids = [str(a.id) for a in daily_articles]
    data = np.mean(model.docvecs[ids],axis=0)                                     # Aggregate method.
    logreturn = next(record.logreturn for record in sp500 if record.date >= date) # sp500 already sorted!
    return Sample(date=date,data=data,logreturn=logreturn)

samples = Namedtuples([create_sample(*grp) for grp in groupby(all_articles,key=lambda article: article.date)])
shuffle(samples)

In [17]:
train_size = int(0.8 * len(samples))
train_samples = samples[:train_size]
test_samples = samples[train_size:]

From this point on we want to compute the $\hat q$ for our problem, and see how it compares with a test set. 

In [19]:
β = 1
r_threshold = 60
u = ut.LinearPlateauUtility(β,r_threshold)

In [30]:
x = np.array(train_samples.datas)
r = np.array(train_samples.logreturns)

x_test = np.array(test_samples.datas)
r_test = np.array(test_samples.logreturns)

In [52]:
problem = pr.Problem(x,r,λ=0,u=u)
problem.solver = cvx.SCS
problem.solve()

-21.191753524944179

In [53]:
problem.outsample_cost(x_test,r_test)

5.9651561073655079

In [54]:
np.linalg.norm(problem.q)

2861845.5229043858